# Importing some packages

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

# Loading data

In [2]:
train_df = pd.read_excel('Train.xlsx')
test_df = pd.read_excel('Test.xlsx')
sub_df = pd.read_csv('Submission.csv')